In [1]:
from src.flight_analysis.scrape import Scrape

In [2]:
flights = Scrape("Frosinone", "Sydney", "2023-10-25")
flights.run_scrape()

In [3]:
flights.data

,departure_datetime,arrival_datetime,airlines,travel_time,origin,destination,layover_n,layover_time,layover_location,price_eur,price_trend,price_value,access_date,one_way,has_train,days_advance
0,2023-10-25 12:05:00,2023-10-27 10:45:00,"[Wizz Air, Air Arabia, AirAsia, AirAsia X]",1 days 13:40:00,FCO,SYD,3,NaT,"[AUH, MAA, KUL]",490,low,466,2023-08-28 18:26:12.504804,True,False,57
1,2023-10-25 21:10:00,2023-10-27 10:00:00,[China Eastern],1 days 03:50:00,FCO,SYD,1,0 days 05:55:00,[PVG],695,low,466,2023-08-28 18:26:12.504822,True,False,58
2,2023-10-25 12:05:00,2023-10-28 08:20:00,"[Wizz Air, Etihad, VietJet Air]",2 days 11:15:00,FCO,SYD,3,NaT,"[AUH, BOM, SGN]",415,low,466,2023-08-28 18:26:12.504825,True,False,57
3,2023-10-25 12:05:00,2023-10-28 08:20:00,"[Wizz Air, Air India, VietJet Air]",2 days 11:15:00,FCO,SYD,3,NaT,"[AUH, BOM, SGN]",424,low,466,2023-08-28 18:26:12.504827,True,False,57
4,2023-10-25 07:00:00,2023-10-27 10:15:00,"[Vueling, XiamenAir]",1 days 18:15:00,FCO,SYD,3,NaT,"[BCN, AMS, XMN]",566,low,466,2023-08-28 18:26:12.504829,True,False,57
5,2023-10-25 14:15:00,2023-10-27 10:45:00,"[Kuwait Airways, AirAsia X]",1 days 11:30:00,FCO,SYD,2,NaT,"[KWI, KUL]",597,low,466,2023-08-28 18:26:12.504832,True,False,57
6,2023-10-25 08:30:00,2023-10-27 10:15:00,"[ITA, XiamenAir]",1 days 16:45:00,FCO,SYD,2,NaT,"[AMS, XMN]",612,low,466,2023-08-28 18:26:12.504834,True,False,57
7,2023-10-25 14:40:00,2023-10-27 10:15:00,"[ITA, XiamenAir]",1 days 10:35:00,FCO,SYD,2,NaT,"[AMS, XMN]",642,low,466,2023-08-28 18:26:12.504836,True,False,57
8,2023-10-25 20:10:00,2023-10-27 11:00:00,"[Turkish Airlines, Malaysia Airlines]",1 days 05:50:00,FCO,SYD,2,NaT,"[IST, KUL]",923,low,466,2023-08-28 18:26:12.504838,True,False,58
9,2023-10-25 08:40:00,2023-10-27 09:15:00,"[ITA, Vietnam Airlines]",1 days 15:35:00,FCO,SYD,2,NaT,"[CDG, SGN]",937,low,466,2023-08-28 18:26:12.504841,True,False,57


In [4]:
flights._url

'https://www.google.com/travel/flights?q=Flights%20to%20Sydney%20from%20Frosinone%20on%202023-10-25%20oneway&curr=EUR&gl=IT'

In [1]:
import pandas as pd
from tqdm import tqdm
from private import private
from src.flight_analysis.database import Database
import src.flight_analysis.apis.flightconnections as fc

db = Database(
    db_host=private.DB_HOST,
    db_name=private.DB_NAME,
    db_user=private.DB_USER,
    db_pw=private.DB_PW,
)

x = db.download_data_connections()
x

 50%|█████     | 1878/3754 [07:55<08:05,  3.86it/s]